In [1]:
import psycopg2
import math
from random import randint


In [3]:
with open("USA_states.txt") as f:
    states = f.readlines()
states = [x.strip() for x in states]

with open("state_codes.txt") as f:
    state_codes = f.readlines()
state_codes = [x.strip() for x in state_codes]

# create sate table
state_table = []
for state in states:
    state_info = {"state_name": state}
    state_table.append(state_info) 
    
state_table_t = tuple(state_table)
#state_table_t

In [4]:
#Create entries for customer table customer name = State_code+number(0:10000)
customer_list = []
customer_table = []
CUST_SIZE = 10
padding = int(math.log10(CUST_SIZE))
for i in range (CUST_SIZE):
    for state_code in state_codes:
        customer_list.append(state_code+str(i).zfill(padding))
        cust_info = {
            "customer_name" : state_code+str(i).zfill(padding),
            "state_id"      : randint(2,len(state_codes))
        }
        customer_table.append(cust_info)

customer_table_t = tuple(customer_table)

In [5]:
#create entries for sales category table
category_table = []
category_list = []
CAT_SIZE = 10
padding = int(math.log10(CAT_SIZE))
for i in range (CAT_SIZE):
    category_list.append("CAT"+str(i).zfill(padding))
    cat_info = {
        "category_name" : "CAT"+str(i).zfill(padding),
        "category_desc" : "Product Category "+str(i).zfill(padding)
    }
    category_table.append(cat_info)

category_table_t = tuple(category_table)

In [6]:
len(category_table_t)

10

In [7]:
#create sales table. 1000 products in each category
product_table = []
product_list = []
PROD_SIZE = 10
padding = int(math.log10(PROD_SIZE))
for cat in category_list:
    for i in range (PROD_SIZE):
        product_list.append("Prod"+str(i).zfill(padding)+cat)
        prod_info = {
            "product_name" : "Prod"+str(i).zfill(padding)+cat,
            "category_id" : category_list.index(cat)+1,
            "list_price"   : float(category_list.index(cat)+10)
        }
        product_table.append(prod_info)

product_table_t = tuple(product_table)

In [8]:
len(product_list)

100

In [9]:
#create sale database
sale_table = []
sale_list = []

for i in range(1000):
    sale_info = {
        "customer_id" : randint(2, CUST_SIZE),
        "product_id"  :  randint(2, PROD_SIZE),
        "quantity"    : randint(1,9),
        "price"       : float(randint(1,50)+10)
    }
    sale_table.append(sale_info)

sale_table_t = tuple(sale_table)

In [10]:
len(sale_table_t)

1000

In [11]:
sale_table_t[10]

{'customer_id': 10, 'price': 11.0, 'product_id': 7, 'quantity': 8}

In [25]:
# create a connection to database. make sure that the db name and password are correct
try:
    conn = psycopg2.connect("dbname='sales' user='postgres' host='/tmp/' password='Paolino1'")
except:
    print "unable to connect to the database"
    
    
conn.set_session(autocommit = True)
cur = conn.cursor()

In [26]:
cur.executemany("""INSERT INTO sales.state(state_name) VALUES (%(state_name)s)""", state_table_t)

In [27]:
cur.executemany("""INSERT INTO sales.customer(customer_name,state_id) VALUES (%(customer_name)s, %(state_id)s)""",
                customer_table_t)


In [28]:
cur.executemany("""INSERT INTO sales.category(category_name,category_desc) VALUES (%(category_name)s, %(category_desc)s)""",
                category_table_t)

In [29]:
cur.executemany("""INSERT INTO sales.product(product_name,category_id,list_price) VALUES (%(product_name)s, %(category_id)s, %(list_price)s)""",
                product_table_t)

In [30]:
cur.executemany("""INSERT INTO sales.sale(customer_id,product_id,quantity,price) VALUES (%(customer_id)s, %(product_id)s, %(quantity)s, %(price)s)""",
                sale_table_t)

In [31]:
try:
    cur.execute("""SELECT * from sales.sale limit 10""")
except:
    print "Failed SELECT from sales.sale"

rows = cur.fetchall()
for row in rows:
    print "   ", row

    (1, 7, 3, 4, Decimal('39.0'))
    (2, 5, 4, 7, Decimal('36.0'))
    (3, 5, 5, 8, Decimal('38.0'))
    (4, 9, 4, 9, Decimal('56.0'))
    (5, 9, 2, 7, Decimal('59.0'))
    (6, 6, 6, 8, Decimal('58.0'))
    (7, 3, 4, 4, Decimal('30.0'))
    (8, 5, 6, 7, Decimal('55.0'))
    (9, 3, 2, 4, Decimal('30.0'))
    (10, 10, 5, 5, Decimal('37.0'))


In [32]:

try:
    cur.execute("""SELECT * from sales.customer limit 5""")
except:
    print "Failed SELECT from sales.customer"

rows = cur.fetchall()
for row in rows:
    print "   ", row

    (1, 'AL0', 40)
    (2, 'AK0', 31)
    (3, 'AZ0', 12)
    (4, 'AR0', 46)
    (5, 'CA0', 31)


In [33]:
#close the connection to db
conn.close()